In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.metrics import f1_score
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Conv1D, MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.regularizers import l2
from sklearn.model_selection import train_test_split
from keras import utils

In [2]:
# for local
# processed_directory = '../processed_data/'

# for google drive
import sys, os
if 'google.colab' in sys.modules:
    # mount google drive
    from google.colab import drive
    drive.mount('/content/gdrive')
    path_to_file = '/content/gdrive/My Drive/cs4248-project/'
    print(path_to_file)
    # move to Google Drive directory
    os.chdir(path_to_file)
    !pwd

Mounted at /content/gdrive
/content/gdrive/My Drive/cs4248-project/
/content/gdrive/My Drive/cs4248-project


In [3]:
data_path = '/content/gdrive/My Drive/cs4248-project/Ailanthus'
train_df = pd.read_csv(data_path+'/train_cleaned_common.csv')
test_df = pd.read_csv(data_path+'/test_cleaned_common.csv')

In [4]:
train_df.head()

,id,text,rating,label
0,0,bromwell high cartoon comedy ran time programs...,9,1
1,10000,homelessness george carlin stated issue years ...,8,1
2,10001,brilliant overacting lesley ann warren best dr...,10,1
3,10002,easily underrated film inn brooks cannon sure ...,7,1
4,10003,typical mel brooks film much less slapstick mo...,8,1


In [5]:
test_df.head()

,id,text,rating,label
0,0,went saw movie last night coaxed friends mine ...,10,1
1,10000,actor turned director bill paxton follows prom...,7,1
2,10001,recreational knowledge sport history pleased d...,9,1
3,10002,saw film sneak preview delightful cinematograp...,8,1
4,10003,bill paxton taken true story 1913 golf open ma...,8,1


In [6]:
mapping = {1: 0, -1: -1}

train_x = train_df["text"]
train_y = train_df.replace({'label': mapping})
train_y = train_y["label"]

test_x = test_df["text"]
test_y = test_df.replace({'label': mapping})
test_y = test_y["label"]

In [7]:
x_train, x_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.2, random_state=0)

In [8]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)

In [9]:
train_y

0        0
1        0
2        0
3        0
4        0
        ..
24995   -1
24996   -1
24997   -1
24998   -1
24999   -1
Name: label, Length: 25000, dtype: int64

In [10]:
sequences = tokenizer.texts_to_sequences(x_train)

In [13]:
# pad sequence to longest sequence
train_x_arr = [sentence.split() for sentence in x_train]
longest_seq = max(len(arr) for arr in train_x_arr)

print(longest_seq)

1342


In [14]:
num_classes = 2

X_train = pad_sequences(sequences, maxlen=longest_seq, padding='post')
Y_train = tf.keras.utils.to_categorical(y_train, num_classes)

In [15]:
X_train

array([[  248,   191,  5398, ...,     0,     0,     0],
       [    5,  1294,   441, ...,     0,     0,     0],
       [  696,     2, 16618, ...,     0,     0,     0],
       ...,
       [  472,   661,   901, ...,     0,     0,     0],
       [  590, 26085, 10477, ...,     0,     0,     0],
       [ 1711,    11,   691, ...,     0,     0,     0]], dtype=int32)

In [16]:
Y_train

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [17]:
sequences_test_subset = tokenizer.texts_to_sequences(x_test)

X_test_subset = pad_sequences(sequences_test_subset, maxlen=longest_seq, padding='post')
Y_test_subset = tf.keras.utils.to_categorical(y_test, num_classes)

In [18]:
sequences_test = tokenizer.texts_to_sequences(test_x)

X_test = pad_sequences(sequences_test, maxlen=longest_seq, padding='post')
Y_test = tf.keras.utils.to_categorical(test_y, num_classes)

In [19]:
vocab_size = len(tokenizer.word_index) + 1

In [20]:
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=longest_seq))
model.add(Conv1D(filters=16, kernel_size=4, activation="relu"))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(10, activation="relu"))
model.add(Dense(2, activation="softmax"))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1342, 100)         3888600   
                                                                 
 conv1d (Conv1D)             (None, 1339, 16)          6416      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 669, 16)          0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 10704)             0         
                                                                 
 dense (Dense)               (None, 10)                107050    
                                                                 
 dense_1 (Dense)             (None, 2)                 22        
                                                        

In [21]:
history = model.fit(X_train, Y_train, epochs=10, batch_size=20)

Epoch 1/10
1000/1000 [==============================] - 26s 13ms/step - loss: 0.3562 - accuracy: 0.8328
Epoch 2/10
1000/1000 [==============================] - 13s 13ms/step - loss: 0.1117 - accuracy: 0.9597
Epoch 3/10
1000/1000 [==============================] - 13s 13ms/step - loss: 0.0286 - accuracy: 0.9915
Epoch 4/10
1000/1000 [==============================] - 13s 13ms/step - loss: 0.0053 - accuracy: 0.9985
Epoch 5/10
1000/1000 [==============================] - 13s 13ms/step - loss: 0.0020 - accuracy: 0.9997
Epoch 6/10
1000/1000 [==============================] - 13s 13ms/step - loss: 0.0117 - accuracy: 0.9969
Epoch 7/10
1000/1000 [==============================] - 13s 13ms/step - loss: 0.0110 - accuracy: 0.9967
Epoch 8/10
1000/1000 [==============================] - 13s 13ms/step - loss: 0.0081 - accuracy: 0.9978
Epoch 9/10
1000/1000 [==============================] - 14s 14ms/step - loss: 9.0306e-04 - accuracy: 0.9998
Epoch 10/10
1000/1000 [==============================] - 13s

In [22]:
loss, accuracy = model.evaluate(X_test_subset, Y_test_subset)

print('Test Accuracy: %f' % (accuracy*100))

157/157 [==============================] - 1s 5ms/step - loss: 1.0013 - accuracy: 0.8688
Test Accuracy: 86.879998


In [23]:
loss, accuracy = model.evaluate(X_test, Y_test)

print('Test Accuracy: %f' % (accuracy*100))

782/782 [==============================] - 5s 6ms/step - loss: 1.1175 - accuracy: 0.8484
Test Accuracy: 84.840000


In [24]:
def get_verdict(x):
    if x == 'A':
        return 0
    else:
        return -1

y_pred = model.predict(X_test_subset)
df = pd.DataFrame(y_pred, columns = ['A', 'B'])

y_pred = pd.DataFrame.idxmax(df, axis=1)
y_pred = y_pred.apply(lambda x: get_verdict(x)).to_frame()

y_pred = y_pred.to_numpy().flatten()

y_pred

score = f1_score(y_test, y_pred, average='macro')
print('f1 score = {}'.format(score))

f1 score = 0.8687916026625704


In [25]:
y_pred = model.predict(X_test)
df = pd.DataFrame(y_pred, columns = ['A', 'B'])

y_pred = pd.DataFrame.idxmax(df, axis=1)
y_pred = y_pred.apply(lambda x: get_verdict(x)).to_frame()

y_pred = y_pred.to_numpy().flatten()

y_pred

score = f1_score(test_y, y_pred, average='macro')
print('f1 score = {}'.format(score))

f1 score = 0.8483547189617144
